In [1]:
import numpy as np
import yfinance as yf
from statsmodels.tsa.stattools import adfuller
from scipy.stats import f
import pandas as pd
import itertools
from statsmodels.tsa.arima.model import ARIMA
import warnings
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_arch
from arch import arch_model
from scipy.stats import chi2
warnings.filterwarnings("ignore")

In [2]:
df = yf.Ticker("^GSPC").history(period="max")
df['daily_log_returns'] = np.log(df['Close'] / df['Close'].shift(1))
df = df.dropna()

log_returns = df['daily_log_returns']


event_windows = [
    ("dot_com_bubble",   "1997-01-01", "2000-03-10", "2003-01-01"),
    ("financial_crisis", "2006-01-01", "2008-09-15", "2011-01-01"),
    ("flash_crash",      "2008-05-06", "2010-05-06", "2012-05-06"),
    ("covid_crash",      "2018-03-11", "2020-03-11", "2022-03-11"),
    ("russia_invasion",  "2020-02-24", "2022-02-24", "2024-02-24"),
]

structured_periods = []
for name, start, event_date, end in event_windows:
    period_data = log_returns.loc[start:end]
    structured_periods.append({
        "event": name,
        "start": start,
        "event_date": event_date,
        "end": end,
        "returns": period_data
    })

for p in structured_periods:
    print(f"{p['event']} → {p['start']} to {p['end']} (event at {p['event_date']})")
    print(f"  Observations: {len(p['returns'])}\n")

dot_com_bubble → 1997-01-01 to 2003-01-01 (event at 2000-03-10)
  Observations: 1509

financial_crisis → 2006-01-01 to 2011-01-01 (event at 2008-09-15)
  Observations: 1259

flash_crash → 2008-05-06 to 2012-05-06 (event at 2010-05-06)
  Observations: 1009

covid_crash → 2018-03-11 to 2022-03-11 (event at 2020-03-11)
  Observations: 1009

russia_invasion → 2020-02-24 to 2024-02-24 (event at 2022-02-24)
  Observations: 1008



In [3]:
p_values = range(4)
q_values = range(4)

arch_lm_results = []
for name, start, event_date, end in event_windows:
    data = log_returns.loc[start:end]

    best_aic = np.inf
    best_order = None
    best_model = None

    for p, q in itertools.product(p_values, q_values):
        try:
            model = ARIMA(data, order=(p, 0, q)).fit()
            if model.aic < best_aic:
                best_aic = model.aic
                best_order = (p, q)
                best_model = model
        except:
            continue

    if best_model is None:
        arch_lm_results.append({
            "event": name,
            "p": None,
            "q": None,
            "LM_stat": None,
            "p_value": None,
            "ARCH_effects": "Model fitting failed"
        })
        continue

    #ARCH-LM test
    residuals = best_model.resid
    lm_test = het_arch(residuals)
    LM_stat = lm_test[0]
    LM_pvalue = lm_test[1]

    arch_lm_results.append({
        "event": name,
        "p": best_order[0],
        "q": best_order[1],
        "LM_stat": LM_stat,
        "p_value": LM_pvalue,
        "ARCH_effects": "Yes" if LM_pvalue < 0.05 else "No"
    })

arch_lm_df = pd.DataFrame(arch_lm_results)

In [4]:
garch_results = []

for _, row in arch_lm_df.iterrows():
    event = row["event"]
    p_val = row["p"]
    q_val = row["q"]

    # Skip if ARMA order not found
    if pd.isnull(p_val) or pd.isnull(q_val):
        continue

    p_val = int(p_val)
    q_val = int(q_val)

    (__, start, event_date, end) = next(x for x in event_windows if x[0] == event)
    data = log_returns.loc[start:end]

    try:
        arma_model = ARIMA(data, order=(p_val, 0, q_val)).fit()
        residuals = arma_model.resid
    except:
        continue

    best_aic = np.inf
    best_bic = np.inf
    best_aic_model = None
    best_bic_model = None
    best_aic_order = None
    best_bic_order = None

    for gp, gq in itertools.product(range(1, 4), range(1, 4)):
        try:
            garch_mod = arch_model(residuals, vol='Garch', p=gp, q=gq, mean='Zero')
            garch_fit = garch_mod.fit(disp="off")

            if garch_fit.aic < best_aic:
                best_aic = garch_fit.aic
                best_aic_order = (gp, gq)
                best_aic_model = garch_fit

            if garch_fit.bic < best_bic:
                best_bic = garch_fit.bic
                best_bic_order = (gp, gq)
                best_bic_model = garch_fit
        except:
            pass

    garch_results.append({
        "event": event,
        "ARMA(p,q)": (p_val, q_val),
        "GARCH(p,q) AIC": best_aic_order,
        "AIC": best_aic,
        "GARCH(p,q) BIC": best_bic_order,
        "BIC": best_bic,
        "model_AIC": best_aic_model,
        "model_BIC": best_bic_model
    })

garch_summary_df = pd.DataFrame([{
    "event": r["event"],
    "ARMA(p,q)": r["ARMA(p,q)"],
    "Best GARCH(p,q) AIC": r["GARCH(p,q) AIC"],
    "AIC": r["AIC"],
    "Best GARCH(p,q) BIC": r["GARCH(p,q) BIC"],
    "BIC": r["BIC"]
} for r in garch_results])

print("=== GARCH summary results ===")
print(garch_summary_df)
print()


=== GARCH summary results ===
              event ARMA(p,q) Best GARCH(p,q) AIC          AIC  \
0    dot_com_bubble    (0, 0)              (2, 3) -8896.810455   
1  financial_crisis    (3, 0)              (3, 1) -7677.692660   
2       flash_crash    (2, 0)              (3, 3) -5855.022640   
3       covid_crash    (3, 2)              (2, 1) -6475.197944   
4   russia_invasion    (0, 3)              (1, 2) -6199.471489   

  Best GARCH(p,q) BIC          BIC  
0              (1, 1) -8871.086035  
1              (3, 1) -7652.002295  
2              (2, 1) -5828.006177  
3              (1, 1) -6458.179392  
4              (1, 2) -6179.808595  



/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 8. The message is:
Positive directional derivative for linesearch
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 8. The message is:
Positive directional derivative for linesearch
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 8. The message is:
Positive directional derivative for linesearch
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(


In [11]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from arch import arch_model
from scipy.stats import chi2

# Wald test function
def garch_wald_test(garch_before, garch_after):
    b1 = garch_before.params
    b2 = garch_after.params
    V1 = garch_before.param_cov
    V2 = garch_after.param_cov


    diff = b1 - b2
    V = V1 + V2
    W = diff.values.T @ np.linalg.inv(V.values) @ diff.values
    df = len(diff)
    p = 1 - chi2.cdf(W, df)

    return {
        "Wald_stat": W,
        "df": df,
        "p_value": p,
        "significant": p < 0.05,
        "param_diff": diff
    }

# Run Wald tests
wald_results = []

for g in garch_results:
    event_name = g["event"]
    garch_order = g["GARCH(p,q) AIC"]
    arma_order = g["ARMA(p,q)"]

    if garch_order is None or arma_order is None:
        continue

    # Get full period and split point
    period = next(p for p in structured_periods if p["event"] == event_name)
    full_data = period["returns"]
    before_data = full_data.loc[:period["event_date"]]
    after_data = full_data.loc[period["event_date"]:]


    try:
        # Fit ARMA on both periods
        arma_before = ARIMA(before_data, order=(arma_order[0], 0, arma_order[1])).fit()
        arma_after = ARIMA(after_data, order=(arma_order[0], 0, arma_order[1])).fit()

        resid_before = arma_before.resid
        resid_after = arma_after.resid

        # Fit GARCH(p, q) on both
        garch_before = arch_model(resid_before, vol="Garch", p=garch_order[0], q=garch_order[1], mean="Zero").fit(disp="off")
        garch_after = arch_model(resid_after, vol="Garch", p=garch_order[0], q=garch_order[1], mean="Zero").fit(disp="off")

        # Run Wald test
        wald_result = garch_wald_test(garch_before, garch_after)

        wald_results.append({
            "event": event_name,
            "ARMA(p,q)": arma_order,
            "GARCH(p,q)": garch_order,
            **wald_result
        })

    except Exception as e:
        wald_results.append({
            "event": event_name,
            "error": str(e)
        })

# Convert results to DataFrame
wald_df = pd.DataFrame(wald_results)

# Print results
for result in wald_results:
    print(f"\n================== {result['event'].upper()} ==================")

    if 'error' in result:
        print(f"Error: {result['error']}")
        continue

    print(f"Significant difference in GARCH parameters? → {'Yes' if result['significant'] else 'No'}")
    print(f"Wald statistic: {result['Wald_stat']:.4f}, p-value: {result['p_value']}")
    print("\nParameter Differences:")
    print(result["param_diff"])



================== DOT_COM_BUBBLE ==================
Significant difference in GARCH parameters? → Yes
Wald statistic: 32.5231, p-value: 1.295036102999525e-05

Parameter Differences:
omega      -0.000006
alpha[1]   -0.009218
alpha[2]    0.014101
beta[1]    -0.027344
beta[2]    -0.027344
beta[3]     0.064028
Name: params, dtype: float64

================== FINANCIAL_CRISIS ==================
Significant difference in GARCH parameters? → Yes
Wald statistic: 181415896.5412, p-value: 0.0

Parameter Differences:
omega      -0.000006
alpha[1]   -0.034285
alpha[2]   -0.034285
alpha[3]   -0.034285
beta[1]     0.102335
Name: params, dtype: float64

================== FLASH_CRASH ==================
Significant difference in GARCH parameters? → Yes
Wald statistic: 9415088.8297, p-value: 0.0

Parameter Differences:
omega       0.000002
alpha[1]    0.066667
alpha[2]   -0.147619
alpha[3]    0.018105
beta[1]     0.260000
beta[2]    -0.094245
beta[3]    -0.068952
Name: params, dtype: float64

=======